In [ ]:
pip install pyzbar opencv-python
pip install paho-mqtt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pyzbar import pyzbar
import paho.mqtt.client as mqtt
import cv2

In [ ]:
#Informacion imagen
class InfoImg:
    def __init__(self, tipo, datos):
        self.tipo = tipo
        self.datos = datos

In [ ]:
def decode(image):
    # Decodes all barcodes from an image
    decoded_objects = pyzbar.decode(image)

    if not decoded_objects:
        print("No barcodes detected.")
        return None  # Retorna None si no hay códigos de barras

    # Procesar solo el primer código de barras detectado
    obj = decoded_objects[0]
    print("Detected barcode:", obj)
    image = draw_barcode(obj, image)

    # Decodificar el tipo y los datos
    tipo = obj.type
    datos = obj.data.decode("utf-8")  # Decodificar los datos a texto
    print("Type:", tipo)
    print("Data:", datos)
    print()

    # Retornar los datos como un número (si es posible)
    try:
        return int(datos)  # Intenta convertir los datos a un número entero
    except ValueError:
        return datos  # Si no es un número, retorna los datos como texto

    # Retornar la lista de instancias de infoImg
    return info_list

def draw_barcode(decoded, image):
    # n_points = len(decoded.polygon)
    # for i in range(n_points):
    #     image = cv2.line(image, decoded.polygon[i], decoded.polygon[(i+1) % n_points], color=(0, 255, 0), thickness=5)
    # uncomment above and comment below if you want to draw a polygon and not a rectangle
    image = cv2.rectangle(image, (decoded.rect.left, decoded.rect.top), 
                            (decoded.rect.left + decoded.rect.width, decoded.rect.top + decoded.rect.height),
                            color=(0, 255, 0),
                            thickness=5)
    return image

def publicador(infoImg):
    #Caso 1: Primero en recibir el codigo
    #Caso 2: intermedio
    #Caso 3: volver a crear el codigo de barra y enviarlo
    # al correo del profersor
client = mqtt.Client()
client.connect("test.mosquitto.org", 1883, 60)
client.publish("PATRONES2025/G1","hola")

In [ ]:
if __name__ == "__main__":
    from glob import glob

    barcodes = glob("Caso5.png")

    for barcode_file in barcodes:
        # Cargar la imagen con OpenCV
        img = cv2.imread(barcode_file)

        # Decodificar el primer código de barras detectado
        decoded_data = decode(img)

        if decoded_data is not None:
            print(f"Datos del código de barras: {decoded_data}")
        else:
            print("No se detectaron códigos de barras.")

Detected barcode: Decoded(data=b'312546789760', type='DATABAR_EXP', rect=Rect(left=17, top=8, width=650, height=111), polygon=[Point(x=17, y=9), Point(x=17, y=119), Point(x=262, y=119), Point(x=507, y=118), Point(x=667, y=116), Point(x=667, y=8)], quality=141, orientation='UP')
Type: DATABAR_EXP
Data: 312546789760

312546789760
